In [17]:
from yahoofinancials import YahooFinancials

In [1]:
from sqlalchemy import create_engine
import db_params

import pandas as pd

db_kwargs = db_params.get_db_params(
        db_name="gwgdb", 
        db_type="postgresql"
    )

In [61]:
import psycopg2
from psycopg2 import Error

In [2]:
symbol = "NK"

In [ ]:
engine = create_engine(f'postgresql://{db_kwargs["user"]}:{db_kwargs["password"]}@{db_kwargs["host"]}:{db_kwargs["port"]}/{db_kwargs["database"]}')

In [62]:
connection = psycopg2.connect(**db_kwargs)

In [ ]:
insert_query = f""" 
    INSERT INTO ticker (symbol,sector,industry,website,num_emp,profile) 
    VALUES ('{symbol}', '{sector}', '{industry}','{website}', num_emp, '{profile}');
"""

In [28]:
def quote_tickers(tickers):
    return f"{tuple([f'{t}' for t in tickers])}"

In [29]:
quote_tickers(['A','B'])

"('A', 'B')"

In [48]:
symbols = ['A','MRTS','PLUG']
table_name = "ticker"
select_sql = f"""select 
    symbol --,sector,industry,website,num_emp,profile 
    from {table_name}
    where symbol in {quote_tickers(symbols)}
    --order by sector,industry
    ;
"""

In [49]:
print(select_sql)

select 
    symbol --,sector,industry,website,num_emp,profile 
    from ticker
    where symbol in ('A', 'MRTS', 'PLUG')
    --order by sector,industry
    ;



In [50]:
query_results = engine.execute(select_sql).fetchall()

In [51]:
query_results

[('A',), ('PLUG',)]

In [52]:
symbols_db = [i[0] for i in query_results]

In [53]:
symbols_db

['A', 'PLUG']

In [54]:
symbols_new = list(set(symbols).difference(set(symbols_db)))
symbols_new

['MRTS']

In [ ]:
connection = psycopg2.connect(**db_kwargs)

In [67]:
cursor = connection.cursor()

In [68]:
symbols = ['A','MRTS','PLUG']

In [69]:
select_sql = f"""select 
    symbol --,sector,industry,website,num_emp,profile 
    from ticker
    where symbol in {quote_tickers(symbols)}
    --order by sector,industry
    ;
"""

In [71]:
cursor.execute(select_sql)
query_results = cursor.fetchall()

In [72]:
query_results

[('A',), ('PLUG',)]

In [80]:
symbols_new = ['MRTS']

In [85]:
symbol = 'MRTX'

In [86]:
prof_data = YahooFinancials(symbol).get_stock_profile_data()[symbol]

In [87]:
prof_data

{'zip': '92121',
 'sector': 'Healthcare',
 'fullTimeEmployees': 111,
 'compensationRisk': 8,
 'auditRisk': 2,
 'longBusinessSummary': 'Mirati Therapeutics, Inc., a clinical-stage oncology company, develops product candidates to address the genetic and immunological promoters of cancer in the United States. The company develops MRTX849, a KRAS G12C inhibitor, which is in Phase I/II clinical trial for treating non-small cell lung (NSCL), colorectal, pancreatic, and other cancers; and sitravatinib, a spectrum-selective kinase inhibitor that is in Phase II clinical trial for the treatment of NSCL cancer, as well as a KRAS G12D inhibitor program, which is in preclinical development. It has a collaboration and license agreement with BeiGene, Ltd. to develop, manufacture, and commercialize sitravatinib; and a clinical collaboration agreement with Novartis Pharmaceuticals Corporation. Mirati Therapeutics, Inc. was founded in 1995 and is headquartered in San Diego, California.',
 'city': 'San D

In [81]:
for symbol in symbols_new:
    sector = industry = website = profile = num_emp = None
    prof_data = YahooFinancials(symbol).get_stock_profile_data()[symbol]
    if prof_data:
        sector,industry,website, profile, num_emp = \
            prof_data["sector"], prof_data["industry"], prof_data["website"], prof_data["longBusinessSummary"], prof_data["fullTimeEmployees"]

        print(f"sector={sector}")
        if profile or sector:
            try:
                insert_query = f""" 
                    INSERT INTO ticker (symbol,sector,industry,website,num_emp,profile) 
                    VALUES ('{symbol}', '{sector}', '{industry}','{website}', num_emp, '{profile}')
                    ;
                """    
                cursor.execute(insert_query)
                connection.commit()
            except Exception as ex:
                print(str(ex))

TypeError: 'NoneType' object is not subscriptable

In [88]:
def get_sector_profiles(symbols, connection):
    """
        Get sector,profile info from yfinance for missing symbol in local DB, 
        Add missing info to DB
        Query DB for all symbols, sort by sector,industry
    """

    cursor = connection.cursor()
    
    select_sql = f"""select 
        symbol --,sector,industry,website,num_emp,profile 
        from ticker
        where symbol in {quote_tickers(symbols)}
        --order by sector,industry
        ;
    """
    cursor.execute(select_sql)
    query_results = cursor.fetchall()
    symbols_db = [i[0] for i in query_results]
    symbols_new = list(set(symbols).difference(set(symbols_db)))
    print(f"symbols_new = {symbols_new}")

    for symbol in symbols_new:
        sector = industry = website = profile = num_emp = None
        try:
            prof_data = YahooFinancials(symbol).get_stock_profile_data()[symbol]
        except Exception as ex:
            print(str(ex))

        if prof_data:
            sector,industry,website, profile, num_emp = \
                prof_data["sector"], prof_data["industry"], prof_data["website"], prof_data["longBusinessSummary"], prof_data["fullTimeEmployees"]

            print(f"sector={sector}")
            if profile or sector:
                try:
                    insert_query = f""" 
                        INSERT INTO ticker (symbol,sector,industry,website,num_emp,profile) 
                        VALUES ('{symbol}', '{sector}', '{industry}','{website}', num_emp, '{profile}')
                        ;
                    """    
                    cursor.execute(insert_query)
                    connection.commit()
                except Exception as ex:
                    print(str(ex))

    select_sql2 = f"""select 
        symbol,sector,industry,website,num_emp,profile 
        from ticker
        where symbol in {quote_tickers(symbols)}
        order by sector,industry,symbol
        ;
    """
    cursor.execute(select_sql2)
    query_results = cursor.fetchall()

    return query_results

In [89]:
symbols = ['A','AMTX','PLUG']
results = get_sector_profiles(symbols, connection)

symbols_new = []


In [90]:
results

[('AMTX',
  'Energy',
  'Oil & Gas Refining & Marketing',
  'http://www.aemetis.com',
  160,
  "Aemetis, Inc. operates as a renewable fuels and bio-chemicals company in North America and India. The company focuses on the acquisition, development, and commercialization of various technologies that replace traditional petroleum-based products primarily through the conversion of ethanol and biodiesel plants into advanced bio refineries. It owns and operates an ethanol facility in the California Central Valley near Modesto; and a renewable chemical and advanced fuel production facility on the East Coast of India. The company sells biodiesel and refined glycerin to transport companies, resellers, distributors, and refiners through its sales force and independent sales agents, as well as to brokers who resell the product to end-users. It also produces ethanol; and wet distiller's grains, distiller's corn oil, and condensed distillers solubles to dairies and feedlots as animal feed. In additi

In [21]:
symbol,sector,industry,website,profile,num_emp

('ORCL',
 'Technology',
 'Software—Infrastructure',
 'http://www.oracle.com',
 "Oracle Corporation provides products and services that address enterprise information technology environments worldwide. The company's cloud and license business engages in the sale, marketing, and delivery of its applications and infrastructure technologies through cloud and on-premise deployment models, including cloud services and license support; and cloud license and on-premise license. Its cloud software as a service offerings include a suite of cloud software applications, including enterprise resource planning (ERP), enterprise and performance management, supply chain management, human capital management, and customer experience cloud-based industry solutions, as well as NetSuite application suite, a cloud-based ERP solution. The company also provides cloud infrastructure as a service; enterprise database; database products, including MySQL, Oracle TimesTen In-Memory Database, Oracle Berkeley DB, an